### Notebook for Openroute service testings

From the summary_df.csv,which summaries coordinates for every nearby amentity for every property;

Calculate the Travel Distance, Travel Time take to amentities

Also get the Average ,including the average counts for each suburb based on the calculation

Debugged with Chatgpt

In [43]:
import openrouteservice as ors
import folium 
import json
import csv
import time

In [44]:
# Note you have to create an account and get your own personal API key
client = ors.Client(key = "5b3ce3597851110001cf62486cd961fa15f646888b22ae2353c038a9") 

In [47]:
# Open the csv and process for every row of address
with open('properties_stats.csv', mode='r', newline='', encoding='utf-8') as csv_file:
    reader = csv.DictReader(csv_file)
    rows = list(reader)

    # Add new 'Travel Time (minutes)' and 'Travel Distance (km)' columns to the header
    fieldnames = reader.fieldnames + ['Travel Time (minutes)', 'Travel Distance (km)']

    # Iterate over each row and calculate travel time and distance, update the CSV in place
    with open('properties_stats.csv', mode='w', newline='', encoding='utf-8') as updated_csv_file:
        writer = csv.DictWriter(updated_csv_file, fieldnames=fieldnames)
        writer.writeheader()

        for row in rows:
            try:
                # Get coordinates and calculate travel time and distance
                lat1, lon1 = float(row['Latitude']), float(row['Longitude'])
                lat2, lon2 = float(row['Location Latitude']), float(row['Location Longitude'])
                coords = [[lon1, lat1], [lon2, lat2]]

                # Call OpenRouteService API to get route information
                route = client.directions(coordinates=coords, profile='driving-car', format='geojson')
                
                # Extract travel time in minutes
                travel_time_minutes = route['features'][0]['properties']['summary']['duration'] / 60
                
                # Extract travel distance in kilometers
                travel_distance_km = route['features'][0]['properties']['summary']['distance'] / 1000

                # Update the row with travel time and distance
                row['Travel Time (minutes)'] = f"{travel_time_minutes:.2f}"
                row['Travel Distance (km)'] = f"{travel_distance_km:.2f}"

                print(f"Processed travel time and distance for {row['Address']} to {row['Location Name']}: {travel_time_minutes:.2f} minutes, {travel_distance_km:.2f} km")

            except Exception as e:
                print(f"Error processing row {row['Address']} to {row['Location Name']}: {e}")
                row['Travel Time (minutes)'] = 'Error'
                row['Travel Distance (km)'] = 'Error'

            # Write updated row with the new columns
            writer.writerow(row)
            time.sleep(2)

Processed travel time and distance for 1210/639 Lonsdale Street, Melbourne VIC 3000 to CBD: 3.24 minutes, 1.29 km
Processed travel time and distance for 1210/639 Lonsdale Street, Melbourne VIC 3000 to Ozford College: 1.82 minutes, 0.61 km
Processed travel time and distance for 1210/639 Lonsdale Street, Melbourne VIC 3000 to Simonds Catholic College St Mary's 7–10 Campus: 3.60 minutes, 1.59 km
Processed travel time and distance for 1210/639 Lonsdale Street, Melbourne VIC 3000 to Haileybury College City Campus: 1.91 minutes, 0.70 km
Processed travel time and distance for 1210/639 Lonsdale Street, Melbourne VIC 3000 to Monument Park: 4.29 minutes, 1.64 km
Processed travel time and distance for 1210/639 Lonsdale Street, Melbourne VIC 3000 to Victoria Green: 4.25 minutes, 1.85 km
Processed travel time and distance for 1210/639 Lonsdale Street, Melbourne VIC 3000 to Unnamed Location: 1.77 minutes, 0.85 km
Processed travel time and distance for 1210/639 Lonsdale Street, Melbourne VIC 3000 to 

# Get the Average For Suburbs

Output the aerage count,travel time and distance in every suburb

In [53]:
import pandas as pd

# Load the CSV file
df_suburb = pd.read_csv('properties_stats.csv')

# Count the amentities for every property 
for i in range(len(df_suburb) - 1):
    # Compare the current row with the next row for the specified columns
    if (df_suburb.loc[i, 'Postcodes'] == df_suburb.loc[i+1, 'Postcodes'] and 
        df_suburb.loc[i, 'Address'] == df_suburb.loc[i+1, 'Address'] and
        df_suburb.loc[i, 'Location Type'] == df_suburb.loc[i+1, 'Location Type']):
        # Set the 'Count' of the next row to 0 if the condition is met
        df_suburb.loc[i+1, 'Count'] = 0

df_suburb['Travel Distance (km)'] = pd.to_numeric(df_suburb['Travel Distance (km)'], errors='coerce')

# Delete rows where "Travel Distance (km)" is too large(than 3km) to avoid huge bias
df_suburb = df_suburb[df_suburb['Travel Distance (km)'] <= 3]




In [56]:
import pandas as pd

# Convert 'Travel Distance (km)' and 'Travel Time (minutes)' to numeric, handling errors
df_suburb['Travel Distance (km)'] = pd.to_numeric(df_suburb['Travel Distance (km)'], errors='coerce')
df_suburb['Travel Time (minutes)'] = pd.to_numeric(df_suburb['Travel Time (minutes)'], errors='coerce')

# Calculate unique address counts per postcode
address_count_by_postcode = df_suburb.groupby('Postcodes')['Address'].nunique().reset_index()
address_count_by_postcode.columns = ['Postcodes', 'Unique Address Count']

# get the total count, average travel distance, and average travel time
df_suburb_postcodes = df_suburb.groupby(['Postcodes', 'Location Type']).agg(
    Count=('Count', 'sum'),
    Average_Distance=('Travel Distance (km)', 'mean'),
    Average_Time=('Travel Time (minutes)', 'mean')
).reset_index()

# Merge with address count to calculate average count
df_suburb_postcodes  = pd.merge(df_suburb_postcodes , address_count_by_postcode, on='Postcodes')
df_suburb_postcodes ['Average Count'] = df_suburb_postcodes ['Count'] / df_suburb_postcodes ['Unique Address Count']

# Drop unnecessary columns for final output
final_average_data = df_suburb_postcodes [['Postcodes', 'Location Type', 'Count', 'Average Count', 'Average_Distance', 'Average_Time']]


final_average_data.to_csv('suburbs_stats.csv',index=False)

In [4]:
# Save the map file (open route services)

# _map.save("../plots/flinders_to_Uni_map.html")
# _map